In [ ]:
import polars as pl
import numpy as np
from faker import Faker
import random
from datetime import datetime, timedelta
from tqdm import tqdm
import itertools
import time

# Iniciar Faker
fake = Faker('es_CO')
np.random.seed(42)
random.seed(42)

# --- Parámetros ---
n_ventas = 150000
n_clientes = 1500
n_canales = 5
n_promociones = 50
n_regiones = 5

start_date_data = datetime(2023, 1, 1)
end_date_data = datetime(2025, 5, 31)

# Parámetros de Elasticidad
elasticidades_por_categoria = {
    'Agua': {'min': -0.5, 'max': -0.8},
    'Gaseosa': {'min': -1.0, 'max': -1.2},
    'Jugo': {'min': -0.6, 'max': -0.95},
    'Bebida de Té': {'min': -1.0, 'max': -1.5},
    'Bebida Energética': {'min': -1.5, 'max': -2.5}
}
prob_elasticidad_positiva = 0.05
rango_elasticidad_positiva = {'min': 0.05, 'max': 0.2}

# --- Parámetros de Control de Fluctuación ---
MAX_PERCENT_CHANGE_PER_MONTH_PRODUCT_AGGREGATE = 0.12
MIN_QUANTITY_PER_SALE = 24
MAX_QUANTITY_PER_SALE = 120
DAILY_QUANTITY_FLUCTUATION_FACTOR = 0.05

# --- Ajuste de Temporalidad ---
seasonal_multipliers = {
    1: random.uniform(0.95, 1.00),
    2: random.uniform(0.95, 1.00),
    3: random.uniform(0.95, 1.00),
    4: random.uniform(0.95, 1.00),
    5: random.uniform(0.95, 1.00),
    6: random.uniform(1.35, 1.50),
    7: random.uniform(1.50, 1.65),
    8: random.uniform(1.45, 1.60),
    9: random.uniform(0.95, 1.00),
    10: random.uniform(0.95, 1.00),
    11: random.uniform(1.00, 1.10),
    12: random.uniform(1.50, 1.70)
}

# --- Parámetros deImpacto en las Promociones ---
PROMOTION_ELASTICITY_MULTIPLIER = 1.5
MIN_PROMOTION_BOOST = 1.2
MAX_PROMOTION_BOOST = 1.8

# --- 1. Tabla Región---
ciudades_colombia = [
    {"nombre_region": "Cundinamarca", "ciudad": "Bogotá", "latitud": 4.7110, "longitud": -74.0721},
    {"nombre_region": "Antioquia", "ciudad": "Medellín", "latitud": 6.2442, "longitud": -75.5812},
    {"nombre_region": "Valle del Cauca", "ciudad": "Cali", "latitud": 3.4516, "longitud": -76.5320},
    {"nombre_region": "Atlántico", "ciudad": "Barranquilla", "latitud": 10.9685, "longitud": -74.7813},
    {"nombre_region": "Bolívar", "ciudad": "Cartagena", "latitud": 10.3910, "longitud": -75.4794}
]

regiones_data = []
for i in tqdm(range(n_regiones), desc="Generando regiones", mininterval=1):
    region_info = ciudades_colombia[i] if i < len(ciudades_colombia) else random.choice(ciudades_colombia)
    regiones_data.append({
        'region_id': i + 1,
        'nombre_region': region_info['nombre_region'],
        'ciudad': region_info['ciudad'],
        'latitud': region_info['latitud'],
        'longitud': region_info['longitud']
    })
df_regiones = pl.DataFrame(regiones_data)

# Pesos de las regiones para la selección de clientes (simulando una distribución poblacional)
pesos_regiones = {
    "Bogotá": 0.20, "Medellín": 0.25, "Cali": 0.15, "Barranquilla": 0.15,
    "Cartagena": 0.10, "Bucaramanga": 0.05, "Cúcuta": 0.04, "Ibagué": 0.03
}
prob_regiones_para_seleccion = np.array([pesos_regiones.get(row['ciudad'], 0.01) for row in df_regiones.iter_rows(named=True)])
prob_regiones_para_seleccion /= prob_regiones_para_seleccion.sum()
regiones_para_seleccion = df_regiones['region_id'].to_list()

# --- 2. Tabla Clientes ---
ciudades_regiones = df_regiones['ciudad'].to_list()
fecha_actual = end_date_data
fecha_365_dias_atras = fecha_actual - timedelta(days=365)

porcentaje_inactivos = random.uniform(0.05, 0.15)
n_clientes_inactivos = int(n_clientes * porcentaje_inactivos)
n_clientes_activos = n_clientes - n_clientes_inactivos

# Mapeo de region_id
ciudad_to_region_id = {row['ciudad']: row['region_id'] for row in df_regiones.iter_rows(named=True)}

clientes = []
for i in tqdm(range(1, n_clientes + 1), desc="Generando clientes", mininterval=1):
    es_inactivo = i <= n_clientes_inactivos
    
    if es_inactivo:
        ultima_compra = fake.date_between(
            start_date=fecha_actual - timedelta(days=730),
            end_date=fecha_365_dias_atras
        )
        base_frecuencia = np.random.randint(1, 5)
    else:
        ultima_compra = fake.date_between(
            start_date=fecha_365_dias_atras,
            end_date=fecha_actual
        )
        base_frecuencia = np.random.randint(5, 15)
        base_frecuencia = min(base_frecuencia, 30)

    ciudad = np.random.choice(ciudades_regiones, p=prob_regiones_para_seleccion)
    region_id = ciudad_to_region_id[ciudad]

    clientes.append({
        'cliente_id': i,
        'nombre': fake.name(),
        'edad': np.random.randint(18, 80),
        'genero': np.random.choice(['M', 'F']),
        'ciudad': ciudad,
        'region_id': region_id,
        'frecuencia_compra': base_frecuencia,
        'ultima_compra': ultima_compra
    })
df_clientes = pl.DataFrame(clientes)

# --- 3. Tabla de Promociones ---
promociones_data = []
for i in tqdm(range(1, n_promociones + 1), desc="Generando promociones", mininterval=1):
    fecha_inicio = fake.date_between(start_date=start_date_data - timedelta(days=180), end_date=end_date_data - timedelta(days=90))
    fecha_fin = fecha_inicio + timedelta(days=random.randint(30, 180))

    if fecha_fin > end_date_data.date():
        fecha_fin = end_date_data.date()

    promociones_data.append({
        'promocion_id': i,
        'nombre_promocion': f"Promo {i}",
        'descuento_porcentaje': np.random.randint(5, 30),
        'fecha_inicio': fecha_inicio,
        'fecha_fin': fecha_fin
    })
df_promociones = pl.DataFrame(promociones_data)

# Actualizando df_clientes con sensibilidad a promociones
avg_promotion_discount = df_promociones['descuento_porcentaje'].mean() / 100
clientes_updated = []
for cliente in tqdm(df_clientes.iter_rows(named=True), desc="Actualizando clientes con promociones", mininterval=1):
    base_frecuencia = cliente['frecuencia_compra']
    if cliente['cliente_id'] > n_clientes_inactivos:
        promotion_sensitivity = 1 + (avg_promotion_discount * random.uniform(0.5, 1.5))
        base_frecuencia = int(base_frecuencia * promotion_sensitivity)
        base_frecuencia = min(base_frecuencia, 30)
    clientes_updated.append({
        **cliente,
        'frecuencia_compra': base_frecuencia
    })
df_clientes = pl.DataFrame(clientes_updated)

# Generar client_id y region_id para mapear tabla de ventas
client_to_region_id = {row['cliente_id']: row['region_id'] for row in df_clientes.select(['cliente_id', 'region_id']).iter_rows(named=True)}

# --- 4. Tabla Productos ---
volumenes_gaseosa_jugo_ml = [250, 600, 1500]
unidades_por_caja_gaseosa_jugo = [1, 6]
volumenes_energia_ml = [250, 500]
unidades_por_caja_energia = [1, 6]
volumenes_agua_ml = [500, 1000, 5000]
unidades_por_caja_agua = [1, 6]
volumenes_te_ml = [300, 500]
unidades_por_caja_te = [1, 6]

categorias_marcas_sabor_base = [
    {'categoria_base': 'Gaseosa', 'sabores': ['Cola', 'Naranja', 'Limón'], 'marcas': ['Zulianita', 'SaborMax', 'Coca-Loca']},
    {'categoria_base': 'Bebida de Té', 'sabores': ['Té Negro', 'Té Verde'], 'marcas': ['Zulianita', 'TeaVida', 'Coca-Loca']},
    #{'categoria_base': 'Jugo', 'sabores': ['Jugo de Naranja', 'Jugo de Manzana'], 'marcas': ['Zulianita', 'FrutaFresca', 'Coca-Loca']},
    {'categoria_base': 'Bebida Energética', 'sabores': ['Booster Guaraná'], 'marcas': ['Zulianita', 'EnergyBoost', 'Coca-Loca']},
    {'categoria_base': 'Agua', 'sabores': ['Con Gas', 'Sin Gas'], 'marcas': ['Zulianita', 'AquaPura', 'Coca-Loca']}
]

producto_nombres_unicos = set()
productos = []
producto_id = 1

for categoria in tqdm(categorias_marcas_sabor_base, desc="Generando productos", mininterval=1, leave=False):
    categoria_base = categoria['categoria_base']
    sabores = categoria['sabores']
    marcas = categoria['marcas']
    
    if categoria_base in ['Gaseosa']:
        volumenes = volumenes_gaseosa_jugo_ml
        unidades_caja = unidades_por_caja_gaseosa_jugo
    elif categoria_base == 'Bebida Energética':
        volumenes = volumenes_energia_ml
        unidades_caja = unidades_por_caja_energia
    elif categoria_base == 'Agua':
        volumenes = volumenes_agua_ml
        unidades_caja = unidades_por_caja_agua
    else:
        volumenes = volumenes_te_ml
        unidades_caja = unidades_por_caja_te

    for marca in marcas:
        combinations = list(itertools.product(sabores, volumenes, unidades_caja))
        
        category_products = 0
        for sabor, volumen_ml_val, unidades_caja_val in combinations:
            if categoria_base == 'Agua':
                nombre_base = f'ACHE2O {sabor}' if marca == 'Zulianita' else f'{sabor} {marca}'
            elif categoria_base == 'Bebida Energética':
                nombre_base = f'Power Cell {sabor}' if marca == 'Zulianita' else f'{sabor} {marca}'
            elif categoria_base == 'Bebida de Té':
                nombre_base = f'{sabor} Zulianita' if marca == 'Zulianita' else f'{sabor} {marca}'
           # elif categoria_base == 'Jugo':
            #    nombre_base = sabor if marca == 'Zulianita' else f'{sabor} {marca}'
            else:
                nombre_base = f'Zulianita {sabor}' if marca == 'Zulianita' else f'{sabor} {marca}'
            
            nombre_producto_str = f'{nombre_base} {volumen_ml_val // 1000}L x {unidades_caja_val}uds' if volumen_ml_val >= 1000 else f'{nombre_base} {volumen_ml_val}mL x {unidades_caja_val}uds'
            
            if nombre_producto_str in producto_nombres_unicos:
                print(f"Advertencia: Nombre duplicado '{nombre_producto_str}' en {categoria_base}, marca {marca}. Saltando...")
                continue
            
            producto_nombres_unicos.add(nombre_producto_str)
            productos.append({
                'producto_id': producto_id,
                'nombre_producto': nombre_producto_str,
                'categoria': categoria_base,
                'marca': marca,
                'volumen_ml_base': volumen_ml_val,
                'unidades_caja_base': unidades_caja_val
            })
            producto_id += 1
            category_products += 1
        
        print(f"Generados {category_products} productos para {categoria_base}, marca {marca}")

df_productos = pl.DataFrame(productos)
n_productos = len(df_productos)
print(f"Total de productos generados: {n_productos}")

# --- 5. Tabla Historial de Precios ---
historico_precios = []
hist_precio_id_counter = 1

precios_por_ml_base_categoria = {
    'Agua': 2.5, 'Gaseosa': 3.5, 'Jugo': 4.0, 'Bebida Energética': 10.0, 'Bebida de Té': 6.0
}
costos_por_ml_base_categoria = {
    'Agua': 1.0, 'Gaseosa': 1.8, 'Jugo': 2.8, 'Bebida Energética': 6.5, 'Bebida de Té': 3.8
}

total_price_records = n_productos * 29
for producto_row in tqdm(df_productos.iter_rows(named=True), desc="Generando historico de precios", total=n_productos, mininterval=1):
    current_date = start_date_data.replace(day=1)
    volumen_para_calculo = producto_row['volumen_ml_base'] if producto_row['volumen_ml_base'] > 0 else 1000
    unidades_para_calculo = producto_row['unidades_caja_base'] if producto_row['unidades_caja_base'] > 0 else 1
    categoria = producto_row['categoria']

    base_precio_ml_actual = precios_por_ml_base_categoria.get(categoria, 3.0)
    base_costo_ml_actual = costos_por_ml_base_categoria.get(categoria, 1.8)

    initial_precio_unitario = round(base_precio_ml_actual * volumen_para_calculo * unidades_para_calculo, 2)
    initial_costo_variable = round(base_costo_ml_actual * volumen_para_calculo * unidades_para_calculo, 2)

    while current_date <= end_date_data:
        historico_precios.append({
            'historico_precio_id': hist_precio_id_counter,
            'producto_id': producto_row['producto_id'],
            'fecha_actualizacion': current_date.date(),
            'precio_base': initial_precio_unitario,
            'costo_variable': initial_costo_variable
        })
        hist_precio_id_counter += 1

        incremento_porcentaje_precio = random.uniform(0.00, 0.015)
        incremento_porcentaje_costo = random.uniform(0.00, 0.015)

        initial_precio_unitario = round(initial_precio_unitario * (1 + incremento_porcentaje_precio), 2)
        initial_costo_variable = round(initial_costo_variable * (1 + incremento_porcentaje_costo), 2)

        if initial_costo_variable >= initial_precio_unitario:
            initial_costo_variable = round(initial_precio_unitario * 0.7, 2)

        if current_date.month == 12:
            current_date = current_date.replace(year=current_date.year + 1, month=1)
        else:
            current_date = current_date.replace(month=current_date.month + 1)

df_historico_precios = pl.DataFrame(historico_precios)

# --- 6. Tabla Canales ---
canales = {
    'canal_id': range(1, n_canales + 1),
    'nombre_canal': ['Supermercado', 'Tienda de Conveniencia', 'E-commerce', 'Farmacia', 'Hipermercado'],
    'tipo_canal': ['Físico', 'Físico', 'Online', 'Físico', 'Físico']
}
df_canales = pl.DataFrame({k: list(v) for k, v in tqdm(canales.items(), desc="Generando canales", total=len(canales), mininterval=1)})

def get_channel_weights(current_date):
    weights = {
        'Supermercado': 0.40,
        'Tienda de Conveniencia': 0.20,
        'E-commerce': 0.15,
        'Farmacia': 0.05,
        'Hipermercado': 0.35
    }
    total_weight = sum(weights.values())
    return {k: v / total_weight for k, v in weights.items()}

# --- 7. Tabla Inventario---
zulianita_product_ids = df_productos.filter(pl.col('marca') == 'Zulianita')['producto_id'].to_list()
n_months = 12
n_inventarios = len(zulianita_product_ids) * n_regiones * n_months

inventarios = []
inventario_id = 1

current_date = end_date_data.replace(day=1)
start_inventory_date = current_date - timedelta(days=365)
inventory_dates = []

while current_date > start_inventory_date:
    last_day = (current_date.replace(month=current_date.month % 12 + 1, day=1) - timedelta(days=1)).date()
    inventory_dates.append(last_day)
    current_date = current_date.replace(month=current_date.month - 1 if current_date.month > 1 else 12, year=current_date.year - 1 if current_date.month == 1 else current_date.year)

total_inventory_combinations = len(zulianita_product_ids) * n_regiones * len(inventory_dates)
for fecha in inventory_dates:
    for region_id in df_regiones['region_id']:
        for producto_id in tqdm(zulianita_product_ids, desc=f"Generando inventario para {fecha}", leave=False, mininterval=1):
            inventarios.append({
                'inventario_id': inventario_id,
                'producto_id': producto_id,
                'region_id': region_id,
                'stock': np.random.randint(50, 500),
                'fecha_actualizacion': fecha
            })
            inventario_id += 1

df_inventarios = pl.DataFrame(inventarios)

# --- 8. Tabla Ventas ---
sales_data_chunks = []
venta_id_counter_final = 1

last_product_monthly_state = {}
monthly_product_target_quantities = {}
daily_product_sales_distribution = {}

product_category_map = {row['producto_id']: row['categoria'] for row in df_productos.select(['producto_id', 'categoria']).iter_rows(named=True)}
product_brand_map = {row['producto_id']: row['marca'] for row in df_productos.select(['producto_id', 'marca']).iter_rows(named=True)}

total_days_simulation = (end_date_data - start_date_data).days + 1

daily_sales_schema = {
    'venta_id': pl.Int64,
    'fecha': pl.String,
    'cliente_id': pl.Int64,
    'producto_id': pl.Int64,
    'cantidad': pl.Int64,
    'canal_id': pl.Int64,
    'region_id': pl.Int64,
    'promocion_id': pl.Int64,
    'historico_precio_id': pl.Int64
}

monthly_sales_trend_multiplier = 1.0
last_month_year = None

# Preómputo de Precios
price_lookup = {}
for prod_id in df_productos['producto_id'].to_list():
    price_records = df_historico_precios.filter(pl.col('producto_id') == prod_id).sort('fecha_actualizacion')
    price_lookup[prod_id] = price_records

sales_generated = 0
promoted_sales_count = 0
total_sales_count = 0

target_promotion_ratio = 0.30  # Start with 30% as a baseline, adjust dynamically
min_promotion_ratio = 0.20
max_promotion_ratio = 0.40

# Cantidades iniciales de producto para contorl de fluctuaciones
initial_avg_quantity = n_ventas / n_productos / total_days_simulation * 30  # Approx monthly average
if marca == 'Zulianita' :
    juice_initial_quantity = 700
else: 
    juice_initial_quantity = 500

for day_offset in tqdm(range(total_days_simulation), desc="Generando datos de ventas", mininterval=1):
    current_sale_date = start_date_data + timedelta(days=day_offset)
    current_sale_date_str = current_sale_date.strftime('%Y-%m-%d')
    current_month_str = current_sale_date.strftime('%Y-%m')

    month = current_sale_date.month
    year = current_sale_date.year

    if current_month_str != last_month_year:
        if last_month_year is not None:
            prev_month_end = current_sale_date - timedelta(days=1)
            prev_month_start = prev_month_end.replace(day=1)
            prev_month_str_for_key = prev_month_start.strftime('%Y-%m')

            df_prev_month_sales_list = []
            for chunk_df in sales_data_chunks:
                chunk_filtered = chunk_df.filter(
                    (pl.col('fecha').str.strptime(pl.Date, '%Y-%m-%d') >= prev_month_start.date()) &
                    (pl.col('fecha').str.strptime(pl.Date, '%Y-%m-%d') <= prev_month_end.date())
                )
                if not chunk_filtered.is_empty():
                    df_prev_month_sales_list.append(chunk_filtered)

            if df_prev_month_sales_list:
                df_prev_month_sales_combined = pl.concat(df_prev_month_sales_list, how="vertical_relaxed")

                df_prev_month_sales_combined = df_prev_month_sales_combined.join(
                    df_historico_precios.select(['historico_precio_id', 'precio_base']),
                    on='historico_precio_id',
                    how='left'
                )

                monthly_summary = df_prev_month_sales_combined.group_by('producto_id').agg(
                    pl.sum('cantidad').alias('cantidad_total'),
                    pl.mean('precio_base').alias('precio_promedio')
                )

                for row in monthly_summary.iter_rows(named=True):
                    key = (row['producto_id'], prev_month_str_for_key)
                    last_product_monthly_state[key] = {
                        'precio': row['precio_promedio'],
                        'cantidad': row['cantidad_total']
                    }

        monthly_product_target_quantities.clear()
        daily_product_sales_distribution.clear()

        days_in_current_month = (current_sale_date.replace(month=current_sale_date.month % 12 + 1, day=1) - timedelta(days=1)).day

        for prod_id in df_productos['producto_id'].to_list():
            prev_month_key = (prod_id, (current_sale_date - timedelta(days=current_sale_date.day)).replace(day=1).strftime('%Y-%m'))

            prev_month_total_quantity = last_product_monthly_state.get(prev_month_key, {'cantidad': 0})['cantidad']
            prev_month_avg_price = last_product_monthly_state.get(prev_month_key, {'precio': 0})['precio']

            if prev_month_total_quantity == 0:
                base_target_quantity = (n_ventas / n_productos / total_days_simulation) * days_in_current_month * 2
                if product_category_map[prod_id] == 'Jugo':
                    base_target_quantity = juice_initial_quantity  # Set initial juice quantity to 700
            else:
                base_target_quantity = max(prev_month_total_quantity, juice_initial_quantity * 0.75)  # Use max to prevent decline propagation

            current_price_record = price_lookup[prod_id].filter(
                pl.col('fecha_actualizacion').cast(pl.Date) <= current_sale_date.date()
            ).sort('fecha_actualizacion', descending=True).head(1)

            current_prod_price = current_price_record['precio_base'].item() if not current_price_record.is_empty() else 100

            porcentaje_cambio_precio = 0
            if prev_month_avg_price > 0:
                porcentaje_cambio_precio = (current_prod_price - prev_month_avg_price) / prev_month_avg_price

            categoria_producto = product_category_map.get(prod_id)
            rango_epd = elasticidades_por_categoria.get(categoria_producto, {'min': -1.0, 'max': -2.0})
            epd_objetivo = random.uniform(rango_epd['min'], rango_epd['max'])
            
            if random.random() < prob_elasticidad_positiva:
                epd_objetivo = random.uniform(rango_elasticidad_positiva['min'], rango_elasticidad_positiva['max'])

            # Cap elasticity impact to prevent excessive decline
            epd_influenced_change = min(epd_objetivo * porcentaje_cambio_precio, 0.0)  # No negative effect beyond zero

            promociones_activas = df_promociones.filter(
                (pl.col('fecha_inicio').cast(pl.Date) <= current_sale_date.date()) &
                (pl.col('fecha_fin').cast(pl.Date) >= current_sale_date.date())
            )

            if not promociones_activas.is_empty():
                descuento_promedio = promociones_activas['descuento_porcentaje'].mean() / 100
                promotion_boost = random.uniform(MIN_PROMOTION_BOOST, MAX_PROMOTION_BOOST)
                epd_influenced_change = epd_influenced_change * PROMOTION_ELASTICITY_MULTIPLIER * (1 + descuento_promedio)
                base_monthly_change_factor = promotion_boost
            else:
                base_monthly_change_factor = random.uniform(0.95, 1.05) * (1 + epd_influenced_change)

            # Cap the negative impact of elasticity
            clipped_base_change_factor = np.clip(
                base_monthly_change_factor,
                0.95,  # Increased minimum to 0.95
                1 + MAX_PERCENT_CHANGE_PER_MONTH_PRODUCT_AGGREGATE
            )

            # Enhanced growth for juices
            if categoria_producto == 'Jugo' and marca == 'Zulianita':
                growth_factor = 1.10  # Increased to 15% monthly growth
            elif categoria_producto == 'Gaseosa' and marca == 'Zulianita':
                growth_factor = 1.15
            elif categoria_producto == 'Bebida Energética' and marca == 'Coca-Loca':
                growth_factor = 1.15
            else:
                growth_factor = 1.05  # 5% for other categories

            current_seasonal_multiplier = seasonal_multipliers.get(month, 1.0)
            
            if categoria_producto in ['Agua', 'Gaseosa', 'Bebida de Té']:
                if month in [6, 7, 8]:
                    current_seasonal_multiplier *= 1.35
            
            if month == 12:
                current_seasonal_multiplier *= 1.35

            final_monthly_change_factor = clipped_base_change_factor * current_seasonal_multiplier * growth_factor

            # Dynamic floor for juices
            target_monthly_quantity = int(base_target_quantity * final_monthly_change_factor)
            if categoria_producto == 'Jugo':
                target_monthly_quantity = max(target_monthly_quantity, int(juice_initial_quantity * 0.85))  # Floor at 85% of 700 (595)
            else:
                target_monthly_quantity = max(target_monthly_quantity, int(initial_avg_quantity * 0.5))  # Floor at 50% for others

            monthly_product_target_quantities[(prod_id, current_month_str)] = target_monthly_quantity

        for prod_id, total_monthly_q in monthly_product_target_quantities.items():
            if prod_id[1] != current_month_str:
                continue

            base_daily_q = total_monthly_q / days_in_current_month

            for d_offset_in_month in range(days_in_current_month):
                date_in_month = current_sale_date.replace(day=1) + timedelta(days=d_offset_in_month)
                daily_q = base_daily_q * random.uniform(1 - DAILY_QUANTITY_FLUCTUATION_FACTOR, 1 + DAILY_QUANTITY_FLUCTUATION_FACTOR)
                daily_product_sales_distribution[(prod_id[0], date_in_month.strftime('%Y-%m-%d'))] = max(1, int(daily_q))

        last_month_year = current_month_str

    base_sales_per_day = n_ventas / total_days_simulation
    overall_daily_sales_num_multiplier = random.uniform(0.95, 1.05)
    num_sales_today = int(base_sales_per_day * monthly_sales_trend_multiplier * overall_daily_sales_num_multiplier)
    num_sales_today = max(1, num_sales_today)

    daily_sales_data = []

    # Use tuple keys consistent with daily_product_sales_distribution
    current_day_product_targets = {
        (prod_id, current_sale_date_str): daily_product_sales_distribution.get((prod_id, current_sale_date_str), 0)
        for prod_id in df_productos['producto_id'].to_list()
    }

    # Use only prod_id for products_for_today_sales
    products_for_today_sales = [
        prod_id[0] for prod_id, target_q in current_day_product_targets.items() if target_q > 0
    ]

    channel_weights = get_channel_weights(current_sale_date)
    canal_ids = df_canales['canal_id'].to_list()
    canal_probs = [channel_weights[canal_name] for canal_name in df_canales['nombre_canal']]

    # Dynamic adjustment of promotion probability
    current_promotion_ratio = promoted_sales_count / total_sales_count if total_sales_count > 0 else 0.50
    adjusted_promotion_prob = np.clip(target_promotion_ratio + (0.50 - current_promotion_ratio) * 0.1, min_promotion_ratio, max_promotion_ratio)

    for _ in range(num_sales_today):
        product_id = None
        if products_for_today_sales:
            # Extract quantities for the current day using prod_id
            remaining_quantities = np.array([
                current_day_product_targets[(p_id, current_sale_date_str)]
                for p_id in products_for_today_sales
            ])
            if remaining_quantities.sum() > 0:
                probabilities = remaining_quantities / remaining_quantities.sum()
                product_id = np.random.choice(products_for_today_sales, p=probabilities)
            else:
                product_id = random.choice(df_productos['producto_id'].to_list())
        else:
            product_id = random.choice(df_productos['producto_id'].to_list())

        cliente_id = np.random.choice(df_clientes['cliente_id'])
        region_id = client_to_region_id[cliente_id]
        canal_id = np.random.choice(canal_ids, p=canal_probs)

        promociones_validas_hoy = df_promociones.filter(
            (pl.col('fecha_inicio').cast(pl.Date) <= current_sale_date.date()) &
            (pl.col('fecha_fin').cast(pl.Date) >= current_sale_date.date())
        )

        promocion_id = None
        if not promociones_validas_hoy.is_empty() and random.random() < adjusted_promotion_prob:
            promocion_id = random.choice(promociones_validas_hoy['promocion_id'].to_list())

        precio_registro = price_lookup[product_id].filter(
            (pl.col('fecha_actualizacion').cast(pl.Date) <= current_sale_date.date())
        ).sort('fecha_actualizacion', descending=True).head(1)

        if precio_registro.is_empty():
            continue

        historico_precio_id = precio_registro['historico_precio_id'].item()

        quantity = random.randint(MIN_QUANTITY_PER_SALE, MAX_QUANTITY_PER_SALE)

        if (product_id, current_sale_date_str) in current_day_product_targets and current_day_product_targets[(product_id, current_sale_date_str)] > 0:
            quantity_for_sale_attempt = random.randint(MIN_QUANTITY_PER_SALE, MAX_QUANTITY_PER_SALE)
            quantity = min(quantity_for_sale_attempt, current_day_product_targets[(product_id, current_sale_date_str)])
            quantity = max(1, quantity)

            current_day_product_targets[(product_id, current_sale_date_str)] -= quantity
            if current_day_product_targets[(product_id, current_sale_date_str)] <= 0:
                if product_id in products_for_today_sales:
                    products_for_today_sales.remove(product_id)

        daily_sales_data.append({
            'venta_id': venta_id_counter_final,
            'fecha': current_sale_date_str,
            'cliente_id': cliente_id,
            'producto_id': product_id,
            'cantidad': quantity,
            'canal_id': canal_id,
            'region_id': region_id,
            'promocion_id': promocion_id,
            'historico_precio_id': historico_precio_id
        })
        venta_id_counter_final += 1
        total_sales_count += 1
        if promocion_id is not None:
            promoted_sales_count += 1

        if venta_id_counter_final > n_ventas:
            print(f"Objetivo de {n_ventas} ventas alcanzado. Deteniendo la generación de datos.")
            break

    if daily_sales_data:
        sales_data_chunks.append(pl.DataFrame(daily_sales_data, schema=daily_sales_schema))
    
    if venta_id_counter_final > n_ventas:
        break


df_ventas = pl.concat([chunk for chunk in tqdm(sales_data_chunks, desc="Concatenando ventas", mininterval=1)], how="vertical_relaxed")
df_ventas = df_ventas.slice(0, n_ventas)

# --- Guardar en CSV ---
df_clientes.write_csv('clientes.csv')
df_productos.write_csv('productos.csv')
df_historico_precios.write_csv('historico_precios.csv')
df_canales.write_csv('canales.csv')
df_regiones.write_csv('regiones.csv')
df_promociones.write_csv('promociones.csv')
df_inventarios.write_csv('inventarios.csv')
df_ventas.write_csv('ventas.csv')

print(f"\nConjuntos de datos generados y guardados como CSV.")
print(f"Total de ventas: {len(df_ventas)}")
print(f"Total de productos: {len(df_productos)}")
print(f"Total de entradas de historial de precios: {len(df_historico_precios)}")
print(f"Total de clientes: {len(df_clientes)}")
print(f"Total de regiones: {len(df_regiones)}")
print(f"Total de canales: {len(df_canales)}")
print(f"Total de promociones: {len(df_promociones)}")
print(f"Total de entradas de inventario: {len(df_inventarios)}")

# --- Verificación de distribución de regiones en clientes ---
print("\nDistribución de regiones en clientes:")
region_dist = df_clientes.group_by('ciudad').agg(pl.count('cliente_id').alias('num_clientes')).sort('num_clientes', descending=True)
for row in region_dist.iter_rows(named=True):
    print(f"{row['ciudad']}: {row['num_clientes']} clientes ({row['num_clientes']/n_clientes*100:.2f}%)")

# --- Verificación de consistencia de region_id en ventas ---
print("\nVerificación de consistencia de region_id en ventas:")
sample_ventas = df_ventas.join(df_clientes.select(['cliente_id', 'region_id']), on='cliente_id', how='left').head(5)
for row in sample_ventas.iter_rows(named=True):
    print(f"Venta ID {row['venta_id']}: Cliente ID {row['cliente_id']}, Venta region_id {row['region_id']}, Cliente region_id {row['region_id_right']}")

# --- Verificación de competidores por categoría ---
print("\nVerificación de competidores por categoría:")
competitor_dist = df_productos.filter(pl.col('marca') != 'Zulianita').group_by('categoria', 'marca').agg(pl.count('producto_id').alias('num_productos')).sort('categoria')
for row in competitor_dist.iter_rows(named=True):
    print(f"Categoría {row['categoria']}, Marca {row['marca']}: {row['num_productos']} productos")

Generando datos de ventas: 100%|██████████| 897/897 [15:12<00:00,  1.02s/it]



Conjuntos de datos generados y guardados como CSV.
Total de ventas: 199909
Total de productos: 40
Total de entradas de historial de precios: 1200
Total de clientes: 2000
Total de regiones: 10
Total de canales: 5
Total de promociones: 20
Total de entradas de inventario: 120

Verificación de variación mensual (primeros 5 productos):
Producto 1:
  2023-1 a 2023-2: -4.87%
  2023-2 a 2023-3: 18.17%
  2023-3 a 2023-4: -12.41%
  2023-4 a 2023-5: 9.22%
  2023-5 a 2023-6: -1.06%
  2023-6 a 2023-7: -12.43%
  2023-7 a 2023-8: -21.36%
  2023-8 a 2023-9: -5.43%
  2023-9 a 2023-10: -5.58%
  2023-10 a 2023-11: 0.35%
  2023-11 a 2023-12: 17.60%
  2023-12 a 2024-1: -13.15%
  2024-1 a 2024-2: 2.67%
  2024-2 a 2024-3: 12.45%
  2024-3 a 2024-4: 1.73%
  2024-4 a 2024-5: 2.91%
  2024-5 a 2024-6: -17.03%
  2024-6 a 2024-7: 1.61%
  2024-7 a 2024-8: -14.62%
  2024-8 a 2024-9: -17.17%
  2024-9 a 2024-10: 28.99%
  2024-10 a 2024-11: -2.98%
  2024-11 a 2024-12: 15.60%
  2024-12 a 2025-1: 0.59%
  2025-1 a 2025-2: